# MagNav.jl Demonstration Notebook
This notebook provides a demonstration of some of the key functions provided in MagNav.jl: https://github.com/MIT-AI-Accelerator/MagNav.jl

Feel free to change any parameters of interest.

## Import packages and DataFrames

The DataFrames listed below provide useful information about the flight data collected by Sander Geophysics Ltd. (SGL) and magnetic anomaly maps.

Dataframe  | Description
:--------- | :----------
`df_map`   | map files relevant for SGL flights
`df_cal`   | SGL calibration flight lines
`df_flight`| SGL flight files
`df_all`   | all flight lines
`df_nav`   | all *navigation-capable* flight lines
`df_event` | pilot-recorded in-flight events

In [1]:
cd(@__DIR__)
# uncomment line below to use local MagNav.jl (downloaded folder)
using Pkg; Pkg.activate("../"); Pkg.instantiate()
using MagNav
using CSV, DataFrames
using Plots: plot, plot!
using Random: seed!
using Statistics: mean, median, std
seed!(33); # for reproducibility
include("dataframes_setup.jl"); # setup DataFrames


  Activating project at `d:\Research\Research Study\06 MagNav Codes\MagNav`
Precompiling project...
           ✗ GDAL_jll
           ✗ GDAL
           ✗ ArchGDAL
           ✗ MagNav
  0 dependencies successfully precompiled in 35 seconds. 521 already precompiled.

The following 2 direct dependencies failed to precompile:

ArchGDAL 

Failed to precompile ArchGDAL [c9ce4bd3-c3d5-55b8-8973-c0e20141b8c3] to "C:\\Users\\liuha\\.julia\\compiled\\v1.11\\ArchGDAL\\jl_5FB2.tmp".
ERROR: LoadError: InitError: could not load library "C:\Users\liuha\.julia\artifacts\a0f65aec92810567654b7ca3b29b0786088806a6\bin\libnetcdf-19.dll"
The specified module could not be found. 
Stacktrace:
  [1] dlopen(s::String, flags::UInt32; throw_error::Bool)
    @ Base.Libc.Libdl .\libdl.jl:120
  [2] dlopen(s::String, flags::UInt32)
    @ Base.Libc.Libdl .\libdl.jl:119
  [3] macro expansion
    @ C:\Users\liuha\.julia\packages\JLLWrappers\GfYNv\src\products\library_generators.jl:63 [inlined]
  [4] __init__()
    @ NetC

ErrorException: Failed to precompile MagNav [f91b31a4-be4d-40e3-b767-4b8c09c10076] to "C:\\Users\\liuha\\.julia\\compiled\\v1.11\\MagNav\\jl_B6B2.tmp".

## Flight line selection

Select Flight 1006 (see [readme](https://github.com/MIT-AI-Accelerator/MagNav.jl/blob/master/readmes/Flt1006_readme.txt)) and gather the [`XYZ20` data structure](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#MagNav.XYZ20), which contains the GPS-based trajectory [`Traj` data structure](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#MagNav.Traj), inertial navigation system [`INS` data structure](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#MagNav.INS), flight information, magnetometer readings, and auxilliary sensor data.

In [ ]:
flight = :Flt1006 # select flight, full list in df_flight
xyz    = get_XYZ(flight,df_flight); # load flight data


The `xyz` flight data struct is of type `MagNav.XYZ20` (for the 2020 SGL flight data collection), which is a subtype of `MagNav.XYZ` (the abstract type for any flight data in MagNav.jl). There are 76 fields, which can be accessed using dot notation. Note that `xyz` holds all the flight data from the HDF5 file, but Boolean indices can be used as a mask to return specific portion(s) of flight data.

In [ ]:
typeof(xyz)


In [ ]:
fieldnames(MagNav.XYZ20)


Here a map is selected, then `df_nav` is filtered into `df_options` to ensure that the selected flight line(s) both corresponds with the selected flight (`:Flt1006`) and exists on the map (`:Eastern_395`). The full list of SGL flights is in `df_flight`, the full list of maps is in `df_map`, and the full list of navigation-capable flight lines is in `df_nav`.

In [ ]:
map_name   = :Eastern_395 # select map, full list in df_map
df_options = df_nav[(df_nav.flight   .== flight  ) .&
                    (df_nav.map_name .== map_name),:]


## Get data for selected flight line

To test navigation, we use Boolean indices (mask) corresponding to flight line 1006.08 in `df_options`.

In [ ]:

line = 1006.08 # select flight line (row) from df_options
ind  = get_ind(xyz,line,df_options); # get Boolean indices


For the Tolles-Lawson calibration, flight line 1006.04 is selected, which occurred at a higher altitude (see [readme](https://github.com/MIT-AI-Accelerator/MagNav.jl/blob/master/readmes/Flt1006_readme.txt)). This is the first calibration box of this flight line. `TL_ind` holds the Boolean indices (mask) just for this portion of the calibration flight line. The full list of calibration flight line options is in `df_cal`.

In [ ]:
TL_i   = 6 # select first calibration box of 1006.04
TL_ind = get_ind(xyz;tt_lim=[df_cal.t_start[TL_i],df_cal.t_end[TL_i]]);


## Plotting

To get an idea of the magnetometer data, we can call some utility functions for plotting.

Note that these are filtered using the `ind` Boolean indices corresponding to the held-out flight `line`.

In [ ]:
show_plot = true
save_plot = false
use_mags  = [:mag_1_uc,:mag_4_uc,:mag_5_uc] # scalar magnetometers to plot

p1 = plot_mag(xyz;ind,show_plot,save_plot, # plot scalar magnetometers
              use_mags     = use_mags,
              detrend_data = true);


In [ ]:
p2 = plot_mag(xyz;ind,show_plot,save_plot, # plot vector magnetometer (fluxgate)
              use_mags     = [:flux_d], # try changing to :flux_a, :flux_b, :flux_c
              detrend_data = true);


Clearly, the in-cabin scalar and vector magnetometers are noisy compared to the stinger magnetometer (Mag 1).

## Create the (linear) Tolles-Lawson model

Next, we select scalar and vector magnetometer readings during the calibration flight and generate the coefficients to perform linear Tolles-Lawson compensation. We are choosing in-cabin scalar magnetometer 4 and vector (flux) magnetometer D. Mag 4 is located on the floor in the rear of the cabin, and Flux D is nearby on the starboard side. Mag 4 is particularly challenging since it contains several 100s to 1000 nT excursions in comparison to the tail stinger.

### Tolles-Lawson calibration

In [ ]:
λ       = 0.025   # ridge parameter for ridge regression
use_vec = :flux_d # selected vector (flux) magnetometer
flux    = getfield(xyz,use_vec) # load Flux D data
TL_d_4  = create_TL_coef(flux,xyz.mag_4_uc,TL_ind;λ=λ); # create Tolles-Lawson coefficients with Flux D & Mag 4


### Tolles-Lawson compensation

In [ ]:
A = create_TL_A(flux,ind)     # Tolles-Lawson `A` matrix for Flux D
mag_1_sgl = xyz.mag_1_c[ind]  # professionally compensated tail stinger, Mag 1
mag_4_uc  = xyz.mag_4_uc[ind] # uncompensated Mag 4
mag_4_c   = mag_4_uc - detrend(A*TL_d_4;mean_only=true); # compensated Mag 4


## Navigation

Here, several steps are done to prepare the flight and map data for the navigation filter. The trajectory (`Traj`) and `INS` data structures are pulled out of the `XYZ20` data structure for convenience, and map data is loaded into a map (`MapS`) data structure. The map is then "upward continued" (via a Fast Fourier Transform) to the trajectory altitude and outfitted with an interpolation function (`itp_mapS`). Finally, the expected scalar magnetometer reading along the flight path (`map_val`) is computed using the interpolation function, as done in `get_map_val()`, then corrected for diurnal effects and the core magnetic field ([IGRF](https://www.ncei.noaa.gov/products/international-geomagnetic-reference-field)).

In [ ]:
traj = get_traj(xyz,ind) # trajectory (GPS) struct
ins  = get_ins( xyz,ind;N_zero_ll=1) # INS struct, "zero" lat/lon to match first `traj` data point
mapS = get_map(map_name,df_map) # load map data
# get map values & map interpolation function
(map_val,itp_mapS) = get_map_val(mapS,traj;return_itp=true)
map_val += (xyz.diurnal + xyz.igrf)[ind] # add in diurnal & core (IGRF)
println("Error for scalar Mag 4: ",round(std(map_val-mag_4_c),digits=2)," nT")


### Create the filter model

Next, we do a rough analysis to estimate appropriate filter parameters, and then we initialize a filter model. Specifically, the autocorrelation between the measured (`mag_4_c`) and true (`map_val`) magnetic values is evaluated. This provides information for setting the magnitudes of the measurement variance `meas_var` and first-order Gauss-Markov (FOGM) standard deviation (`fogm_sigma`) and time constant (`fogm_tau`). These are all approximate, and may not be consistently appropriate for the entire flight line.

In [ ]:
(sigma,tau) = get_autocor(mag_4_c - map_val)


In [ ]:
(P0,Qd,R) = create_model(traj.dt,traj.lat[1];
                         init_pos_sigma = 0.1,
                         init_alt_sigma = 1.0,
                         init_vel_sigma = 1.0,
                         meas_var       = 134^2, # increase for bad mag
                         fogm_sigma     = 134,
                         fogm_tau       = 10.0);


### Run the filter

Here, we use an extended Kalman filter (`:ekf`) and compute:
- `crlb_out`: Cramér–Rao lower bound error
- `ins_out`: error when using the INS alone (dead reckoning)
- `filt_out`: error when using the INS + magnetic measurements (MagNav)

In [ ]:
mag_use = mag_4_c # select Mag 4 for navigation
mag_use .+= map_val[1] - mag_use[1] # remove initial DC offset
(crlb_out,ins_out,filt_out) = run_filt(traj,ins,mag_use,itp_mapS,:ekf;P0,Qd,R,core=true); # run the filter


### Plot the results

Despite the 149 m distance-root-mean-square (DRMS) error, the navigation does not go too far awry for the linear model.

In [ ]:
p3 = plot_map(mapS;legend=false); # plot map background
plot_filt!(p3,traj,ins,filt_out;show_plot=false); # plot GPS, INS (after zeroing), and filter
plot!(p3,legend=:topleft) # move legend as desired


### Corrupting fields

In the below plot, there are some spiky excursions in the TL-compensated magnetometer that do not occur on the map. We can easily compare the two using the `plot_mag_map` convenience function.

In [ ]:
p4 = plot_mag_map(traj,mag_use,itp_mapS) # plot magnetometer vs map


However, current sensors for the 💡 strobe lights 💡 picked some of these up. The current sensors have severe high-frequency noise, so we first low-pass filter them using two convenience functions, `get_bpf` and `bpf_data`.

In [ ]:
lpf     = get_bpf(;pass1=0.0,pass2=0.2,fs=10.0) # get low-pass filter
lpf_sig = -bpf_data(xyz.cur_strb[ind];bpf=lpf)  # apply low-pass filter, sign switched for easier comparison
p5      = plot_basic(xyz.traj.tt[ind],lpf_sig;lab="filtered current for strobe lights"); # plot the low-pass filtered strobe light current sensor


## Training a physics-based neural network architecture

Now we attempt to improve on the prior results by training a neural network that incorporates the Tolles-Lawson (TL) model together with an artificial neural network (NN). Specifically, the NN is provided with cabin current sensor information that produces an additive correction to the TL model. Both the NN parameters and the TL coefficients are trainable, which we designate as model 2c (model 2b, by comparison, keeps the TL coefficients static).

First, we select all available flight data from Flights 1003-1006 (see [readmes](https://github.com/MIT-AI-Accelerator/MagNav.jl/tree/master/readmes)) into `lines_train`, except the held-out flight `line`. The full list of flight lines is in `df_all`.

In [ ]:
flts = [:Flt1003,:Flt1004,:Flt1005,:Flt1006] # select flights for training
df_train = df_all[(df_all.flight .∈ (flts,) ) .& # use all flight data
                  (df_all.line   .!= line),:]    # except held-out line
lines_train = df_train.line # training lines


Here, we add current sensors to the usual inputs, making them available to the neural network. 

In [ ]:
features = [:mag_4_uc, :lpf_cur_com_1, :lpf_cur_strb, :lpf_cur_outpwr, :lpf_cur_ac_lo];


The neural network-based compensation parameters (type of `NNCompParams`) are provided to (and returned by) the training function. These take default values unless they are specified.

In [ ]:
comp_params = NNCompParams(features_setup = features,
                           model_type     = :m2c,
                           y_type         = :d,
                           use_mag        = :mag_4_uc,
                           use_vec        = :flux_d,
                           terms          = [:permanent,:induced,:fdm],
                           terms_A        = [:permanent,:induced,:eddy],
                           sub_diurnal    = true,
                           sub_igrf       = true,
                           bpf_mag        = false,
                           norm_type_A    = :none,
                           norm_type_x    = :standardize,
                           norm_type_y    = :standardize,
                           TL_coef        = TL_d_4,
                           η_adam         = 0.001,
                           epoch_adam     = 300,
                           epoch_lbfgs    = 0,
                           hidden         = [8,4]);


In [ ]:
(comp_params,y_train,y_train_hat,err_train,feats) =
    comp_train(comp_params,lines_train,df_all,df_flight,df_map);


After 300 epochs, model 2c has 37 nT of error on the training data. We next test the performance on the held-out flight `line` using the `comp_test` convenience function. Note that there is also a `comp_train_test` convenience function that does both.

In [ ]:
(_,y_hat,_) =
    comp_test(comp_params,[line],df_all,df_flight,df_map);


We are now in a position to compare the magnetometer readings to the expected map values. The model 2c results ameliorate the signal excursions that are present in the uncompensated and TL-compensated readings. 

Note that the `detrend` function helps remove any persistent bias in the signal, which does not affect the navigation error.

In [ ]:
tt = (xyz.traj.tt[ind] .- xyz.traj.tt[ind][1]) / 60;
p6 = plot(xlab="time [min]", ylab="magnetic field [nT]");
plot!(p6, tt, detrend(map_val  - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="anomaly map");
plot!(p6, tt, detrend(mag_4_uc - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="uncompensated");
plot!(p6, tt, detrend(mag_4_c  - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="Tolles-Lawson");
plot!(p6, tt, detrend(mag_4_uc - y_hat - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="model 2c")


Next, we use the NN output to perform the compensation. Since we selected `y_type = :d` in the NN compensation parameters (`NNCompParams`), we treat the output as the platform component that must be subtracted from the total scalar signal.

In [ ]:
mag_use = mag_4_uc - y_hat # compensate Mag 4 using NN output
mag_use .+= map_val[1] - mag_use[1] # remove initial DC offset
println("TL + NN σ: ",round(std(map_val-mag_use),digits=2))
println("TL σ:      ",round(std(mag_4_c.+(map_val[1]-mag_4_c[1])-map_val),digits=2))
(sigma,tau) = get_autocor(mag_use - map_val)


Finally, we use these results in an extended Kalman filter (EKF) to check navigation performance.

In [ ]:
(P0,Qd,R) = create_model(traj.dt,traj.lat[1];
                         init_pos_sigma = 0.1,
                         init_alt_sigma = 1.0,
                         init_vel_sigma = 1.0,
                         meas_var       = 56^2, # increase for bad mag
                         fogm_sigma     = 56,
                         fogm_tau       = 10);

(crlb_out,ins_out,filt_out) = run_filt(traj,ins,mag_use,itp_mapS,:ekf;P0,Qd,R,core=true);


The physics informed neural network outperforms the linear model!

The notebook continues below to show a similar comparison using a "vanilla" neural network (no linear model). We also provide results for model 2b, which leaves the TL coefficients as constant in the NN learning process. To reproduce the model 2b results, simply replace `model_type = :m2c` with `model_type = :m2b` above and replace the `fogm_sigma` and `fogm_tau` values with the corresponding outputs from `plot_autocor`.

Model             | Mag Error [nT] | Nav Error [m]
----------------- | -------------- | -------
Tolles-Lawson     | 134            | 149 
TL + NN, model 2c | 55             | 74 
TL + NN, model 2b | 39             | 124 
vanilla NN        | 67             | 116 

In [ ]:
p7 = plot_map(mapS;legend=false); # plot map background
plot_filt!(p7,traj,ins,filt_out;show_plot=false); # plot GPS, INS (after zeroing), and filter
plot!(p7,legend=:topleft) # move legend as desired


## Bonus comparison to vanilla neural network

The above approaches used in-cabin vector (Flux D) and scalar (Mag 4) magnetometers, as well as current sensors. Here, we provide the same information to a "vanilla" neural network that does not contain embedded Tolles-Lawson information. We have simply added the vector (fluxgate) magnetometer to the feature list and changed to `model type = :m1`.

In [ ]:
features = [:TL_A_flux_d, :mag_4_uc, :lpf_cur_com_1, :lpf_cur_strb, :lpf_cur_outpwr, :lpf_cur_ac_lo]
comp_params_vanilla = NNCompParams(features_setup = features,
                                   model_type     = :m1,
                                   y_type         = :d,
                                   use_mag        = :mag_4_uc,
                                   terms          = [:permanent,:induced,:fdm],
                                   sub_diurnal    = true,
                                   sub_igrf       = true,
                                   bpf_mag        = false,
                                   norm_type_x    = :standardize,
                                   norm_type_y    = :standardize,
                                   η_adam         = 0.001,
                                   epoch_adam     = 300,
                                   epoch_lbfgs    = 0,
                                   hidden         = [8,4]);


In [ ]:
(comp_params_vanilla,y_train,y_train_hat,err_train,feats) =
    comp_train(comp_params_vanilla,lines_train,df_all,df_flight,df_map);


In [ ]:
(_,y_hat_vanilla,_) =
    comp_test(comp_params_vanilla,[line],df_all,df_flight,df_map);


In [ ]:
mag_use = mag_4_uc - y_hat_vanilla # compensate Mag 4 using NN output
mag_use .+= map_val[1] - mag_use[1] # remove initial DC offset
println("NN σ: ",round(std(map_val-mag_use),digits=2))
println("TL σ: ",round(std(mag_4_c.+(map_val[1]-mag_4_c[1])-map_val),digits=2))
(sigma,tau) = get_autocor(mag_use - map_val)


In [ ]:
tt = (xyz.traj.tt[ind] .- xyz.traj.tt[ind][1]) / 60;
p8 = plot(xlab="time [min]", ylab="magnetic field [nT]");
plot!(p8, tt, detrend(map_val  - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="anomaly map");
plot!(p8, tt, detrend(mag_4_uc - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="uncompensated");
plot!(p8, tt, detrend(mag_4_c  - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="Tolles-Lawson");
plot!(p8, tt, detrend(mag_4_uc - y_hat_vanilla - (xyz.diurnal + xyz.igrf)[ind], mean_only=true), lab="vanilla NN")


In [ ]:
(P0,Qd,R) = create_model(traj.dt,traj.lat[1];
                         init_pos_sigma = 0.1,
                         init_alt_sigma = 1.0,
                         init_vel_sigma = 1.0,
                         meas_var       = 56^2, # increase for bad mag
                         fogm_sigma     = 56,
                         fogm_tau       = 12);

(crlb_out,ins_out,filt_out) = run_filt(traj,ins,mag_use,itp_mapS,:ekf;P0,Qd,R,core=true);


The vanilla neural network *does* do better than Tolles-Lawson alone, but not as well as model 2c.

We are continuing to work on ways to do more with less data.